In [1]:
!git clone https://github.com/YerevaNN/mimic3-benchmarks/

Cloning into 'mimic3-benchmarks'...
remote: Enumerating objects: 1716, done.
remote: Total 1716 (delta 0), reused 0 (delta 0), pack-reused 1716
Receiving objects: 100% (1716/1716), 11.24 MiB | 26.16 MiB/s, done.
Resolving deltas: 100% (1221/1221), done.


In [75]:
!pip install xgboost

     |████████████████████████████████| 166.7 MB 19 kB/s s eta 0:00:01     |███████████████████████████████▍| 163.8 MB 76.3 MB/s eta 0:00:01


In [76]:
import pickle
import pandas as pd
import datetime
from datetime import timedelta
import numpy as np
import matplotlib.pyplot as plt
from timeit import default_timer as timer
from dateutil.relativedelta import relativedelta
def SASDateConvert(inputSeries):
    return pd.DatetimeIndex(pd.to_datetime(inputSeries,format='%d%b%Y:%H:%M:%S.%f')).normalize()

import statsmodels.api as sm

def SASDateNoTimeConvert(inputSeries):
    return pd.DatetimeIndex(pd.to_datetime(inputSeries,format='%d%b%Y')).normalize()


from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split,GridSearchCV, RandomizedSearchCV, StratifiedKFold

from sklearn.ensemble import RandomForestRegressor

import seaborn as sns
import xgboost as xgb
import gc
from sklearn.metrics import mean_squared_error, r2_score 
from sklearn.exceptions import NotFittedError


In [1]:
import pandas as pd

In [14]:
item_map.to_csv('item_map.csv')

In [17]:
import boto3

s3 = boto3.resource('s3')
s3.meta.client.upload_file('item_map.csv', 'aws-athena-query-results-441066871063-us-east-1', 'item_map.csv')

TypeError: upload_file() got an unexpected keyword argument 'header'

In [4]:
import boto3
import io


In [14]:
s3 = boto3.resource('s3')
bucket = s3.Bucket('aws-athena-query-results-441066871063-us-east-1')
prefix_objs = bucket.objects.filter(Prefix="all_columns/part-")
prefix_df = pd.DataFrame()

for obj in prefix_objs:
    key = obj.key
    body = obj.get()['Body'].read()
    temp = pd.read_csv(io.BytesIO(body), encoding='utf8')        
    prefix_df = pd.concat([prefix_df,temp], axis = 0)

In [16]:
prefix_df = prefix_df.reset_index()

In [35]:
prefix_df.to_pickle('patient_data.pkl')

In [22]:
prefix_df.columns

Index(['index', 'SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'LOS', 'ADMITTIME',
       'DISCHTIME', 'DIAGNOSIS', 'AGE', 'Capillary_refill_rate__VALUENUM_min',
       'Capillary_refill_rate__percent_change',
       'Diastolic_blood_pressure__VALUENUM_min',
       'Diastolic_blood_pressure__percent_change',
       'Fraction_inspired_oxygen__VALUENUM_min',
       'Fraction_inspired_oxygen__percent_change',
       'Glascow_coma_scale_eye_opening__VALUENUM_min',
       'Glascow_coma_scale_eye_opening__percent_change',
       'Glascow_coma_scale_motor_response__VALUENUM_min',
       'Glascow_coma_scale_motor_response__percent_change',
       'Glascow_coma_scale_total__VALUENUM_min',
       'Glascow_coma_scale_total__percent_change',
       'Glascow_coma_scale_verbal_response__VALUENUM_min',
       'Glascow_coma_scale_verbal_response__percent_change',
       'Glucose__VALUENUM_min', 'Glucose__percent_change',
       'Heart_Rate__VALUENUM_min', 'Heart_Rate__percent_change',
       'Height__VALUENUM

In [ ]:
def read_icd_diagnoses_table(mimic3_path):
    codes = dataframe_from_csv(os.path.join(mimic3_path, 'D_ICD_DIAGNOSES.csv'))
    codes = codes[['ICD9_CODE', 'SHORT_TITLE', 'LONG_TITLE']]
    diagnoses = dataframe_from_csv(os.path.join(mimic3_path, 'DIAGNOSES_ICD.csv'))
    diagnoses = diagnoses.merge(codes, how='inner', left_on='ICD9_CODE', right_on='ICD9_CODE')
    diagnoses[['SUBJECT_ID', 'HADM_ID', 'SEQ_NUM']] = diagnoses[['SUBJECT_ID', 'HADM_ID', 'SEQ_NUM']].astype(int)
    return diagnoses


In [25]:
prefix_df[['DIAGNOSIS']].head(30)

,DIAGNOSIS
0,SEPSIS;TELEMETRY
1,CORONARY ARTERY DISEASE
2,SEPSIS
3,S/P MOTOR VEHICLE ACCIDENT
4,EXPLORATORY LAPAROTOMY FOR SMALL BOWEL OBSTRUC...
5,CAROTID STENOSIS\CAROTID STENT
6,"PNEUMONIA,SEPSIS"
7,SEPSIS
8,AORTIC STENOSIS\AORTIC VALVE REPLACEMENT; ? CA...
9,AORTIC INSUFFICIENCY\BENTAL PROCEDURE /SDA


In [23]:
prefix_df[['Heart_Rate__VALUENUM_min', 'Heart_Rate__percent_change']].head()


,Heart_Rate__VALUENUM_min,Heart_Rate__percent_change
0,100.0,-0.370000
1,86.0,-0.093023
2,115.0,-0.269565
3,66.0,0.090909
4,82.0,0.060976


In [21]:
prefix_df[prefix_df["Capillary_refill_rate__VALUENUM_min"].notnull()].head()

,index,SUBJECT_ID,HADM_ID,ICUSTAY_ID,LOS,ADMITTIME,DISCHTIME,DIAGNOSIS,AGE,Capillary_refill_rate__VALUENUM_min,...,Separated,Single,Widowed,F,M,Government,Medicaid,Medicare,Private,Self Pay
36,36,1832,176044,272738,7.1729,2148-06-13T18:28:00.000Z,2148-06-20T22:38:00.000Z,CONGESTIVE HEART FAILURE;HYPOTENSION,84.060233,0.0,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN
193,193,9805,188132,259006,2.6471,2130-02-03T09:08:00.000Z,2130-02-09T17:20:00.000Z,GASTROINTESTINAL BLEED,69.106092,0.0,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN
213,213,10468,172565,220776,0.9978,2115-10-29T00:58:00.000Z,2115-10-29T12:00:00.000Z,ABDOMINAL AORTIC ANEURYSM,86.321697,1.0,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN
386,386,18931,117563,292276,2.3834,2174-11-30T11:20:00.000Z,2174-12-06T17:45:00.000Z,MYOCARDIAL INFARCTION\CATH,58.286105,0.0,...,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN
419,419,20575,141116,254624,6.9747,2146-12-02T07:15:00.000Z,2146-12-13T15:55:00.000Z,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,57.002053,0.0,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN


In [96]:
prefix_df.columns

Index(['index', 'SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'LOS', 'ADMITTIME',
       'DISCHTIME', 'DIAGNOSIS', 'AGE', 'Capillary_refill_rate__VALUENUM_min',
       'Capillary_refill_rate__percent_change',
       'Diastolic_blood_pressure__VALUENUM_min',
       'Diastolic_blood_pressure__percent_change',
       'Fraction_inspired_oxygen__VALUENUM_min',
       'Fraction_inspired_oxygen__percent_change',
       'Glascow_coma_scale_eye_opening__VALUENUM_min',
       'Glascow_coma_scale_eye_opening__percent_change',
       'Glascow_coma_scale_motor_response__VALUENUM_min',
       'Glascow_coma_scale_motor_response__percent_change',
       'Glascow_coma_scale_total__VALUENUM_min',
       'Glascow_coma_scale_total__percent_change',
       'Glascow_coma_scale_verbal_response__VALUENUM_min',
       'Glascow_coma_scale_verbal_response__percent_change',
       'Glucose__VALUENUM_min', 'Glucose__percent_change',
       'Heart_Rate__VALUENUM_min', 'Heart_Rate__percent_change',
       'Height__VALUENUM

In [97]:
prefix_df.update(prefix_df[['Asian', 'Black', 'Hispanic', 'Other_Eth', 'Unknown', 'White',
       'Catholic', 'Episcopalian', 'Jewish', 'Other_Rel', 'ProtestantQuaker',
       'Unknown_Rel', 'Divorced', 'Married', 'Other_Mar', 'Separated',
       'Single', 'Widowed', 'F', 'M', 'Government', 'Medicaid', 'Medicare',
       'Private', 'Self Pay']].fillna(0))

In [100]:
prefix_df[['Capillary_refill_rate__VALUENUM_min','Asian', 'Black', 'Hispanic', 'Other_Eth', 'Unknown', 'White',
       'Catholic', 'Episcopalian', 'Jewish', 'Other_Rel', 'ProtestantQuaker',
       'Unknown_Rel', 'Divorced', 'Married', 'Other_Mar', 'Separated',
       'Single', 'Widowed', 'F', 'M', 'Government', 'Medicaid', 'Medicare',
       'Private', 'Self Pay']]

,Capillary_refill_rate__VALUENUM_min,Asian,Black,Hispanic,Other_Eth,Unknown,White,Catholic,Episcopalian,Jewish,...,Separated,Single,Widowed,F,M,Government,Medicaid,Medicare,Private,Self Pay
0,NaN,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1,NaN,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,NaN,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
3,NaN,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,NaN,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28365,NaN,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
28366,NaN,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
28367,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
28368,NaN,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


In [101]:
prefix_df.to_pickle('patient_data.pkl')

In [15]:
hadoop -put item_map.csv /user/livy/item_map.csv

SyntaxError: invalid syntax (<ipython-input-15-44cf5d56e387>, line 1)

In [2]:
item_map = pd.read_csv('mimic3-benchmarks/mimic3benchmark/resources/itemid_to_variable_map.csv')

In [3]:
item_map.head()

,LEVEL2,LEVEL1,ALTERNATIVE,STATUS,STATUS NOTE,ITEMID,MIMIC LABEL,UNITNAME,LINKSTO,COUNT,CATEGORY,CONCEPTID,FLUID,LOINC_CODE,DBSOURCE,Unnamed: 15,PARAM_TYPE,NOTE
0,Alanine aminotransferase,Alanine aminotransferase,ALT,verify,NaN,50861,ALANINE AMINOTRANSFERASE (ALT),NaN,labevents,219475,CHEMISTRY,NaN,BLOOD,6/1/1742,NaN,NaN,NaN,NaN
1,Alanine aminotransferase,Alanine aminotransferase,ALT,verify,NaN,769,ALT,NaN,chartevents,41594,Enzymes,NaN,NaN,NaN,carevue,NaN,NaN,NaN
2,Alanine aminotransferase,Alanine aminotransferase,ALT,verify,NaN,220644,ALT,None,chartevents,37625,Labs,NaN,NaN,NaN,metavision,NaN,Numeric,NaN
3,Albumin,Albumin,NaN,verify,NaN,50862,ALBUMIN,NaN,labevents,146697,CHEMISTRY,NaN,BLOOD,7/1/1751,NaN,NaN,NaN,NaN
4,Albumin,Albumin,NaN,verify,NaN,772,Albumin (>3.2),NaN,chartevents,31022,Chemistry,NaN,NaN,NaN,carevue,NaN,NaN,NaN


In [27]:
!pip install PyAthena

In [28]:
from pyathena import connect
#from pyathena.util import as_pandas
#from __future__ import print_function

# Import libraries
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import boto3
from botocore.client import ClientError
# below is used to print out pretty pandas dataframes
from IPython.display import display, HTML
%matplotlib inline


s3 = boto3.resource('s3')
client = boto3.client("sts")
account_id = client.get_caller_identity()["Account"]
my_session = boto3.session.Session()
region = my_session.region_name
athena_query_results_bucket = 'aws-athena-query-results-'+account_id+'-'+region

try:
    s3.meta.client.head_bucket(Bucket=athena_query_results_bucket)
except ClientError:
    bucket = s3.create_bucket(Bucket=athena_query_results_bucket)
    print('Creating bucket '+athena_query_results_bucket)
cursor = connect(s3_staging_dir='s3://'+athena_query_results_bucket+'/athena/temp').cursor()


# The Glue database name of your MIMIC-III parquet data
gluedatabase="mimiciii"

# location of the queries to generate aline specific materialized views
aline_path = './'

# location of the queries to generate materialized views from the MIMIC code repository
concepts_path = './concepts/'

In [29]:
conn = connect(s3_staging_dir='s3://'+athena_query_results_bucket+'/athena/temp')

In [30]:
query = '''
SELECT * FROM mimiciii.DIAGNOSES_ICD
'''

In [31]:
diagnosis = pd.read_sql(query,conn)

In [33]:
diagnosis.shape

(651047, 5)

In [46]:
diagnosis.columns = diagnosis.columns.str.upper()

In [47]:
diagnosis.columns

Index(['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'SEQ_NUM', 'ICD9_CODE'], dtype='object')

In [56]:
diagnosis['ICD9_CODE3'] = diagnosis['ICD9_CODE'].astype(str).str[0:3]

In [71]:
count_diags = diagnosis.groupby(by=['SUBJECT_ID', 'HADM_ID'])['ICD9_CODE'].nunique().to_frame()
count_diags = count_diags.reset_index(level=['SUBJECT_ID', 'HADM_ID']).rename(columns = {'ICD9_CODE':'ICD9_CODE_COUNT'})

In [72]:
count_diags

,SUBJECT_ID,HADM_ID,ICD9_CODE_COUNT
0,2,163353,3
1,3,145834,9
2,4,185777,9
3,5,178980,3
4,6,107064,8
...,...,...,...
58971,99985,176670,13
58972,99991,151118,17
58973,99992,197084,12
58974,99995,137810,17


In [63]:
diagnosis = diagnosis.sort_values(by=['SUBJECT_ID', 'HADM_ID', 'SEQ_NUM'])

In [78]:
diagnosis.columns

Index(['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'SEQ_NUM', 'ICD9_CODE',
       'ICD9_CODE3'],
      dtype='object')

In [65]:
diagnosis.groupby(by=['SUBJECT_ID', 'HADM_ID'])['ICD9_CODE'].head(1)

608       V3001
611        0389
620         042
629       V3000
632       40391
          ...  
648165     0389
648178    56211
648195     9999
648207     4414
648478    75612
Name: ICD9_CODE, Length: 58976, dtype: object

In [77]:
def one_hot_encode_with_limits(data,object_collist,limit=10):
    '''One Hot Encoding will transform categorical data to numeric, and group those not in the top [10 by default]'''
    for count, column in enumerate(object_collist):
        print (column)
        data[column+"_null"]=pd.isnull(data[column])
        topCols = list(pd.DataFrame(data.groupby(column).size())\
     .sort_values(0,ascending=False).reset_index()[column])[:limit]

        data.loc[data[column].isin(topCols)==False,column]='not_in_top'
        dummies = pd.get_dummies(data[column])
        data[[column+'_'+str(col) for col in list(dummies.columns)]] = dummies


        if column != 'ARRANGEMENT_ID':    
            del data[column]
            
    return data


In [79]:
df1hot = one_hot_encode_with_limits(diagnosis, object_collist = ['ICD9_CODE3'], limit = 60)

ICD9_CODE3


In [83]:
df1hot_groups = df1hot.groupby(['SUBJECT_ID', 'HADM_ID']).max()

In [87]:
df1hot_groups = df1hot_groups.reset_index(level=['SUBJECT_ID', 'HADM_ID'])

KeyError: 'Requested level (SUBJECT_ID) does not match index name (None)'

In [88]:
df1hot_groups = df1hot_groups.drop(columns = ['ROW_ID','ICD9_CODE'])

In [ ]:
df1hot_groups[df1hot_groups.ICD9_CODE3_null == True]

In [93]:
df1hot_groups = df1hot_groups.drop(columns = ['ICD9_CODE3_null'])

In [94]:
df1hot_groups.head()

,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE3_038,ICD9_CODE3_041,ICD9_CODE3_070,ICD9_CODE3_244,ICD9_CODE3_250,ICD9_CODE3_272,ICD9_CODE3_276,...,ICD9_CODE3_E93,ICD9_CODE3_V05,ICD9_CODE3_V10,ICD9_CODE3_V12,ICD9_CODE3_V15,ICD9_CODE3_V29,ICD9_CODE3_V30,ICD9_CODE3_V45,ICD9_CODE3_V58,ICD9_CODE3_not_in_top
0,2,163353,3.0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,1,0,0,0
1,3,145834,9.0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,4,185777,9.0,0,1,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,1
3,5,178980,3.0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,1,0,0,0
4,6,107064,8.0,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,1


In [92]:
df1hot_groups[df1hot_groups.ICD9_CODE3_null == True]

,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE3_null,ICD9_CODE3_038,ICD9_CODE3_041,ICD9_CODE3_070,ICD9_CODE3_244,ICD9_CODE3_250,ICD9_CODE3_272,...,ICD9_CODE3_E93,ICD9_CODE3_V05,ICD9_CODE3_V10,ICD9_CODE3_V12,ICD9_CODE3_V15,ICD9_CODE3_V29,ICD9_CODE3_V30,ICD9_CODE3_V45,ICD9_CODE3_V58,ICD9_CODE3_not_in_top


In [86]:
df1hot_groups.columns

Index(['SUBJECT_ID', 'HADM_ID', 'ROW_ID', 'SEQ_NUM', 'ICD9_CODE',
       'ICD9_CODE3_null', 'ICD9_CODE3_038', 'ICD9_CODE3_041', 'ICD9_CODE3_070',
       'ICD9_CODE3_244', 'ICD9_CODE3_250', 'ICD9_CODE3_272', 'ICD9_CODE3_276',
       'ICD9_CODE3_278', 'ICD9_CODE3_285', 'ICD9_CODE3_287', 'ICD9_CODE3_300',
       'ICD9_CODE3_305', 'ICD9_CODE3_311', 'ICD9_CODE3_348', 'ICD9_CODE3_401',
       'ICD9_CODE3_403', 'ICD9_CODE3_410', 'ICD9_CODE3_412', 'ICD9_CODE3_414',
       'ICD9_CODE3_416', 'ICD9_CODE3_424', 'ICD9_CODE3_427', 'ICD9_CODE3_428',
       'ICD9_CODE3_458', 'ICD9_CODE3_486', 'ICD9_CODE3_493', 'ICD9_CODE3_496',
       'ICD9_CODE3_507', 'ICD9_CODE3_511', 'ICD9_CODE3_518', 'ICD9_CODE3_530',
       'ICD9_CODE3_571', 'ICD9_CODE3_578', 'ICD9_CODE3_584', 'ICD9_CODE3_585',
       'ICD9_CODE3_599', 'ICD9_CODE3_707', 'ICD9_CODE3_733', 'ICD9_CODE3_765',
       'ICD9_CODE3_770', 'ICD9_CODE3_774', 'ICD9_CODE3_780', 'ICD9_CODE3_785',
       'ICD9_CODE3_787', 'ICD9_CODE3_790', 'ICD9_CODE3_995', 'IC

In [66]:
diagnosis.head(30)

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE,ICD9_CODE3
608,1,2,163353,1.0,V3001,V30
609,2,2,163353,2.0,V053,V05
610,3,2,163353,3.0,V290,V29
611,4,3,145834,1.0,0389,038
612,5,3,145834,2.0,78559,785
613,6,3,145834,3.0,5849,584
614,7,3,145834,4.0,4275,427
615,8,3,145834,5.0,41071,410
616,9,3,145834,6.0,4280,428
617,10,3,145834,7.0,6826,682


In [42]:
diagnosis.dtypes

row_id          int64
subject_id      int64
hadm_id         int64
seq_num       float64
icd9_code      object
dtype: object

In [36]:
diagnosis.to_pickle('diagnosis.pkl')

In [102]:
full_df = prefix_df.merge(df1hot_groups, how='left', on=['SUBJECT_ID', 'HADM_ID'])

In [103]:
full_df.shape

(28370, 130)

In [107]:
list(full_df.columns)

['index',
 'SUBJECT_ID',
 'HADM_ID',
 'ICUSTAY_ID',
 'LOS',
 'ADMITTIME',
 'DISCHTIME',
 'DIAGNOSIS',
 'AGE',
 'Capillary_refill_rate__VALUENUM_min',
 'Capillary_refill_rate__percent_change',
 'Diastolic_blood_pressure__VALUENUM_min',
 'Diastolic_blood_pressure__percent_change',
 'Fraction_inspired_oxygen__VALUENUM_min',
 'Fraction_inspired_oxygen__percent_change',
 'Glascow_coma_scale_eye_opening__VALUENUM_min',
 'Glascow_coma_scale_eye_opening__percent_change',
 'Glascow_coma_scale_motor_response__VALUENUM_min',
 'Glascow_coma_scale_motor_response__percent_change',
 'Glascow_coma_scale_total__VALUENUM_min',
 'Glascow_coma_scale_total__percent_change',
 'Glascow_coma_scale_verbal_response__VALUENUM_min',
 'Glascow_coma_scale_verbal_response__percent_change',
 'Glucose__VALUENUM_min',
 'Glucose__percent_change',
 'Heart_Rate__VALUENUM_min',
 'Heart_Rate__percent_change',
 'Height__VALUENUM_min',
 'Height__percent_change',
 'Mean_blood_pressure__VALUENUM_min',
 'Mean_blood_pressure__per

In [104]:
full_df.head()

,index,SUBJECT_ID,HADM_ID,ICUSTAY_ID,LOS,ADMITTIME,DISCHTIME,DIAGNOSIS,AGE,Capillary_refill_rate__VALUENUM_min,...,ICD9_CODE3_E93,ICD9_CODE3_V05,ICD9_CODE3_V10,ICD9_CODE3_V12,ICD9_CODE3_V15,ICD9_CODE3_V29,ICD9_CODE3_V30,ICD9_CODE3_V45,ICD9_CODE3_V58,ICD9_CODE3_not_in_top
0,0,33,176176,296681,1.5542,2116-12-23T22:30:00.000Z,2116-12-27T12:05:00.000Z,SEPSIS;TELEMETRY,82.390144,NaN,...,0,0,0,0,0,0,0,0,0,1
1,1,149,154869,224026,2.6856,2135-02-18T19:16:00.000Z,2135-02-26T17:00:00.000Z,CORONARY ARTERY DISEASE,299.994524,NaN,...,0,0,0,0,0,0,0,0,0,1
2,2,157,107880,264885,4.1192,2106-06-17T19:51:00.000Z,2106-06-24T17:50:00.000Z,SEPSIS,80.533881,NaN,...,0,0,0,0,0,0,0,0,1,1
3,3,183,113341,252525,1.1260,2188-07-03T16:41:00.000Z,2188-07-05T17:30:00.000Z,S/P MOTOR VEHICLE ACCIDENT,51.184120,NaN,...,0,0,0,0,0,0,0,0,0,1
4,4,197,129890,275674,0.8262,2174-03-22T19:47:00.000Z,2174-03-30T18:20:00.000Z,EXPLORATORY LAPAROTOMY FOR SMALL BOWEL OBSTRUC...,45.431896,NaN,...,0,0,0,0,0,0,0,0,0,1


In [108]:
target = full_df['LOS']
variables =full_df.drop(['index', 'SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'LOS', 'ADMITTIME',
       'DISCHTIME', 'DIAGNOSIS'],axis=1)
train_features, test_features, train_labels, test_labels = train_test_split(variables, target, test_size = 0.3, random_state = 1992)


In [109]:
#Define Parameters
from scipy.stats import randint
from scipy.stats import uniform
from sklearn.model_selection import KFold

mcw_high=21
mcw_low=1


mdepth_low=2
mdepth_high=12

#Original List
rlambdas = [0.001,0.01,0.05,0.5,1,10,20]
ralphas = [0.001,0.01,0.05,0.02,0.5,1,10,20]

gamma_high=14
gamma_low=0

#colsam_high=1
#colsam_low=0.5

etas_high=0.4
etas_low=0.01

n_est_high=1000
n_est_low=25

n_iter_search=100
OUT=[]
training_auc=[]
test_auc=[]
oot_auc=[]

top4=[]
j=0


params={'min_child_weight':randint(mcw_low,mcw_high),
     'max_depth':randint(mdepth_low,mdepth_high),
     'reg_lambda':rlambdas,
     #'colsample_bylevel':uniform(colsam_low,colsam_high),
     'learning_rate':uniform(etas_low,etas_high-etas_low),
     'gamma':uniform(gamma_low,gamma_high-gamma_low),
     'reg_alpha':ralphas,
     'n_estimators':randint(n_est_low,n_est_high)   
       }


In [111]:
xgb_reg = xgb.XGBRegressor(n_jobs=4, objective='reg:linear',random_state=1992, importance_type='total_gain')
kfold = KFold(n_splits=5)

In [ ]:
randomized_search = RandomizedSearchCV(xgb_reg, param_distributions=params, n_iter=n_iter_search, cv= kfold, verbose = 2,n_jobs=25, random_state=42)
model5 = randomized_search.fit(train_features,train_labels)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


In [37]:
query_icd = '''
SELECT * FROM mimiciii.D_ICD_DIAGNOSES
'''

In [38]:
icd = pd.read_sql(query_icd,conn)
icd.to_pickle('icd.pkl')

In [39]:
icd.head()

,row_id,icd9_code,short_title,long_title
0,174,01166,TB pneumonia-oth test,"Tuberculous pneumonia [any form], tubercle bac..."
1,175,01170,TB pneumothorax-unspec,"Tuberculous pneumothorax, unspecified"
2,176,01171,TB pneumothorax-no exam,"Tuberculous pneumothorax, bacteriological or h..."
3,177,01172,TB pneumothorx-exam unkn,"Tuberculous pneumothorax, bacteriological or h..."
4,178,01173,TB pneumothorax-micro dx,"Tuberculous pneumothorax, tubercle bacilli fou..."


In [34]:
import pickle